# Import - moduly

In [1]:
import requests
from datetime import datetime
import pandas as pd
import polyline
import geopandas as gpd
from shapely.geometry import LineString
import ast
import json

# Pomocné funkce

In [2]:
def duration(data):
    """
    Vrátí dobu první iterace cesty
    """
    return data['data']['plan']['itineraries'][0]['duration']

def leg_num(data):
    """
    Vrátí počet kroků první iterace cesty
    """
    return len(data['data']['plan']['itineraries'][0]['legs'])    

def start_walk_time(iteration):
    """
    Vrátí dobu chůze z výchozího bodu na první stanici
    """
    return iteration['legs'][0]['duration']

def end_walk_time(iteration):
    """
    Vrátí dobu chůze z poslední stanice do cíle
    """
    return iteration['legs'][-1]['duration']


# Vlastní funkce

In [3]:
# --------------------------------------------- Vytvoření slovníků -----------------------------------------------------------------------------------

def build_route_dict (category, row, name_start:str, name_end:str, date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, municipalities=False, Prague=False):
    """
    Vrátí slovník ve struktuře koncové tabulky (pro obce (default) a pro parkoviště)

    municipalities=True -> vzhled slovníku pro obce
    Prague=True -> slovník pro Prahu   
    
    category - kategorie - podle rozhodnutí, jestli je to nejrychlejší nebo nejméně přestupů
    row - řádek z tabulky, který vznikl z .iterrow()
    name_start - jméno sloupce "výchozí pozice"
    end_start - jméno sloupce "koncové pozice
    date - zvolené datum
    time - zvolený čas
    iteration - je část odpovědi API, která odpovídá jedné iteraci trasy
    iteration_index - index iterace (protože používám j, iteration in enumerate())
    all_coords - GPS body pro celou trasu
    transport_vehicles - seznam jedinečných dopravních prostředků
    itinerary - celý itinerář (slovy)
    geometry_legs - geometrie jednotlivých kroků pro případnou vizualizaci; dá se to celkem jednoduše přes split rozdělit na jednotlivé kroky (oddělovač ;) a prostřed + linestring (oddělovač+)
    
    """
    dictionary = {"KATEGORIE_ITERACE": category}        # nejrychlejsi/nejmene prestupu/rail, pokud nejméně přestupů = nejrychlejší, je jenom nejrychlejší
    if municipalities:
        dictionary["OKRES_KOD"] = row['OKRES_KOD']
    if Prague:
        dictionary["data_address"] = row["data_address"]
        if len(transport_vehicles) > 0:
            dictionary["ZASTAVKA_JMENO"] = iteration['legs'][1]['from']['name']
            dictionary["ZASTAVKA_lat"] = iteration['legs'][1]['from']['lat']
            dictionary["ZASTAVKA_lon"] = iteration['legs'][1]['from']['lon']
        else:
            dictionary["ZASTAVKA_lat"] = None
            dictionary["ZASTAVKA_lon"] = None
    dictionary[name_start] = row[name_start]
    dictionary[name_end] = row[name_end]
    dictionary["DATUM A CAS"] = f'{date}T{time}'           # zadaný datum a čas
    dictionary["SPICKA"] = spicka
    dictionary["DOBA"] = round(iteration['duration']/60) # minuty
    dictionary["CAS_ZACATEK"] = datetime.fromisoformat(iteration['start']).time().isoformat() # kontrolní
    dictionary["CAS_KONEC"] = datetime.fromisoformat(iteration['end']).time().isoformat()
    dictionary["DELKA"] = round(sum(leg['distance'] for leg in iteration['legs'])/1000, 1) # kilometry
    dictionary["POCET_PRESTUPU"] = transfers
    dictionary["DOPRAVNI_PROSTREDKY"] = ', '.join(s for s in transport_vehicles)
    dictionary["TRASA"] = all_coords
    # dictionary["POCET_USEKU"] = len(iteration['legs'])        # v principu duplicita, protože pocet_useku = transfers * 2 + 1 a úseky dávají víc smysl
    dictionary["CISLO_ITERACE"] = iteration_index + 1                 # jen pro kontrolu
    dictionary["KROKY"] = itinerary                              # kontrolní               
    dictionary["LINESTRINGY_ETAPY"] = geometry_legs              # geometrie jednotlivých kroků pro případnou vizualizaci
                                                        # dá se to celkem jednoduše přes split rozdělit na jednotlivé kroky (oddělovač ;) a prostřed + linestring (oddělovač+)
    return dictionary


# ----------------------------------- Rozhodnutí směru -----------------------------------------------------------------------------
def direction_decide(time:str, lat_1:float, lon_1:float, lat_2:float, lon_2:float):
    """
    rozhodne směr cesty podle denní doby, první dvojice je lokace startu pro ráno
    """
# Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
    if datetime.strptime(time, "%H:%M").hour < 12:
        from_lat, from_lon = lat_1, lon_1
        to_lat, to_lon = lat_2, lon_2
        spicka = "ranni"
    else:
        from_lat, from_lon = lat_2, lon_2
        to_lat, to_lon = lat_1, lon_1
        spicka = "odpoledni"
    return from_lat, from_lon, to_lat, to_lon, spicka

# --------------------------------------------- Funkce - přípravy na LineString -------------------------------------------------------------
# -------------------- celková trasa
def build_coordination_string(iteration):
    """
    Převedení geometrie, kterou vrací API, na sérii GPS bodů a vytvoření seznamu pro převedení do LineString geometrie

    Přizpůsobeno pro OTP, all_coords = předpřipravený seznam, do kterého se bude ukládat
    """
    all_coords = []
    for leg in iteration['legs']:
        coords = polyline.decode(leg["legGeometry"]["points"]) # přepsání hrůzostrašných písmen na lat a lon dvojice
        coords = [(lon, lat) for lat, lon in coords] # příprava na vytvoření linestring - protože linestring je lon a lat, tak jen přehození pořadí pro každou dvojici
        if all_coords:
            if coords[0] == all_coords[-1]:
                all_coords.extend(coords[:-1])
            else:
                all_coords.extend(coords)
        else:
            all_coords.extend(coords)
    return all_coords

# ------------------- string jednolivých kroků --------------------------------------------------------------------------------
def build_geometry_legs_string(iteration):
    """
    Vytvoří řetězec ve tvaru "způsob dopravy+série GPS bodů"

    e to připravené na případnou možnost obarvení trasy podle dopravního prostředku
    dá se to celkem jednoduše přes split rozdělit na jednotlivé kroky (oddělovač ,) a prostřed + linestring (oddělovač+) 
    """
    parts = []
    for leg in iteration['legs']:
        coords = polyline.decode(leg["legGeometry"]["points"]) # přepsání hrůzostrašných písmen na lat a lon dvojice
        coords = [(lon, lat) for lat, lon in coords] # příprava na vytvoření linestring - protože linestring je lon a lat, tak jen přehození pořadí pro každou dvojici
        parts.append(f'{leg["mode"]}+{(coords)}')
    return ",".join(parts)

# ---------------------------------------------------- Řetězec itineráře -------------------------------------------------------
def build_itinerary(iteration):
    """
    Vytvoří řetězec celkového itineráře cesty

    Jednolivé kroky mají formát: "způsob dopravy, trvání v min, délka úseku v m ((linka), výchozí stanice - koncová stanice)
    """
    parts = []
    for leg in iteration["legs"]:
        if leg['mode'] == 'WALK': # WALK nemá linku, takže by to házelo chyby 
            part = f"{leg['mode']}, {round(leg['duration']/60)} min, {leg['distance']} m ({leg['from']['name']} - {leg['to']['name']})"
        else:
            part = f"{leg['mode']}, {round(leg['duration']/60)} min, {leg['distance']} m  ({leg['route']['shortName']}, {leg['from']['name']} - {leg['to']['name']})"
        parts.append(part)
    return " + ".join(parts)

# ---------------------------------------------------- Dopravní prostředky ------------------------------------------------------
def build_transport_list(iteration):
    """
    Vrátí seznam jedinečných dopravních prostředků používaných v dané cestě (seřazené podle abecedy) a celkového itineráře (pro vynechání stejných tras, které mají jen jiné čekací doby)
    """
    transports = []
    transport_vehicles = []

    for leg in iteration['legs']:
        if leg['mode'] == 'WALK':
            transports.append(leg['mode'])
        else:
            transports.append(leg['route']['shortName'])
            if leg['mode'] not in transport_vehicles:
                transport_vehicles.append(leg['mode'])
    return transports, sorted(transport_vehicles)




def transfer_num(iteration):
    """
    Vrátí počet přestupů (počet segmentů veřejné dopravy mínus 1)
    """
    PT_num = sum([1 for leg in iteration['legs'] if leg['mode'] != "WALK" ])
    return max(0, PT_num - 1)

## Funkce - export do GeoJsonu

In [6]:
def CSV_GeoJson_export(adress:str, separator:str, GPSColumnName:str, OutputAdress:str):
    """
    Převede seznam GPS na LineString a soubor uloží do GeoJson
    """

    routes = pd.read_csv(adress, sep=separator)
    routes[GPSColumnName] = routes[GPSColumnName].apply(lambda x: LineString(ast.literal_eval(x)))
    routes = gpd.GeoDataFrame(routes, geometry=GPSColumnName)
    routes.to_file(OutputAdress)
    return 


# Společné části kódu

In [17]:
url = "http://localhost:8080/otp/gtfs/v1"
headers = {"Content-Type": "application/json"}

# GraphQL query
query = '''query Parking (
        $from_lat: Float!,              # definování proměnných, které se vkládají ve "variables"
        $from_lon: Float!, 
        $to_lat: Float!, 
        $to_lon: Float!,
        $Date: String!,
        $Time: String!,
        $window: Long!,
        $NumInt: Int!
        ) 
    {
    plan(
        from: {lat: $from_lat, lon: $from_lon}
        to: {lat: $to_lat, lon: $to_lon}
        date: $Date
        time: $Time                    
        searchWindow: $window           # v jakém časovém okně od zadaného času, bude hledat trasy (v s)
        numItineraries: $NumInt         # počet iterací
        transportModes: [               # definice dopravních prostředků
            {mode: WALK},              
            {mode: BUS},               
            {mode: TRAM}, 
            {mode: RAIL}, 
            {mode: SUBWAY}, 
            {mode: TROLLEYBUS}, 
            {mode: FERRY}
        ]
        ) {
            itineraries {
            start                       # datetime, timezone - Prague
            end                         # datetime, timezone - Prague 
            duration                    # in seconds
            legs {
                mode
                route {
                shortName
                }
                distance
                duration
                from {
                name
                }
                to {
                name
                }
                legGeometry { 
                length
                points
                }
            }
            }
        }
        }
'''

# Dates = ["2025-11-03", "2025-11-04", "2025-11-05", "2025-11-06", "2025-11-07"]
Dates = ["2025-11-10", "2025-11-11", "2025-11-12", "2025-11-13", "2025-11-14"]
Times = ["06:30", "07:30", "08:30", "17:00"]

# Kanceláře - parkoviště

In [ ]:
PR_Kancelare = pd.read_csv("kancelare_parkoviste.csv")

routes = []
errors = []
limit_time = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)

for date in Dates:
    for time in Times:
        for _, row in PR_Kancelare.iterrows():

            # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
            from_lat, from_lon, to_lat, to_lon, spicka = direction_decide(time, row["PARKOVISTE_Y"], row["PARKOVISTE_X"], row["KANCELAR_X"], row["KANCELAR_Y"])

            try:
                payload = {
                    "query": query,
                    "variables": {
                        "from_lat": float(from_lat),
                        "from_lon": float(from_lon),
                        "to_lat": float(to_lat),
                        "to_lon": float(to_lon),
                        "Date": date,
                        "Time": time,
                        "window": 1800,
                        "NumInt": 5
                        },
                    "operationName": "Parking"
                    }
                
                # volání API
                response = requests.post(url, headers=headers, json=payload, timeout=30)
                data = response.json()
                print(f'{_}, {row['PARKOVISTE']}, {row['KANCELAR']}, {date}, {time}')
                

                # ------------------------TRANSFORMACE-------------------------

                # pomocné proměnné
                
                duration_fast = duration(data)
                leg_num_low = leg_num(data)

                fast_route = {}
                leg_low_route = {}

                for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
                    all_coords = build_coordination_string(iteration)
                    geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

                    itinerary = build_itinerary(iteration)

                    transport_lines, transport_vehicles = build_transport_list(iteration)
                    # transports - konkrétní linky
                    # transport_vehicles - typy dopravních prostředků
                    transfers = transfer_num(iteration)
                    
                    # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
                    time_walk_start = start_walk_time(iteration)
                    time_walk_end = end_walk_time(iteration)

                    # ------------ FAST ----------
                    # rozhodovací bloky - rozdělení na nejrychlejší a s nejméně přestupy
                    if int(iteration['duration']) < duration_fast or iteration_index == 0:
                        duration_fast = int(iteration['duration'])
                        fast_transports = transport_lines
                        fast_route = build_route_dict('nejrychlejsi', row, "PARKOVISTE", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs)

                    # -------------- LOW LEG NUMBER --------------------
                    # tady je to jedno, ale v případě obcí je třeba ošetřit cestu na vlak (ta může být delší a pak se to dá oseknout)
                    if len(iteration['legs']) < leg_num_low and transfers > 0:
                        # počet kroků, kde je více než 2 vynechá iterace, kde je jen chůze
                        if time_walk_start <= limit_time and time_walk_end <= limit_time: 
                            # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                            leg_num_low = len(iteration['legs'])
                            leg_low_route = build_route_dict ("nejmene prestupu", row, "PARKOVISTE", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs)
                
                if leg_low_route:
                    if fast_route["POCET_PRESTUPU"] == leg_low_route["POCET_PRESTUPU"]: 
                        routes.append(fast_route)
                    else:
                        routes.append(fast_route)
                        routes.append(leg_low_route)
                else:
                    routes.append(fast_route)
            except Exception:
                error_dict = {
                    "PARKOVISTE": row["PARKOVISTE"],
                    "KANCELAR": row["KANCELAR"],
                    "DATUM": date,
                    "CAS": time,
                    "from_lat": from_lat,
                    "from_lon":from_lon,
                    "to_lat": to_lat,
                    "to_lon": to_lon,
                    "data": data
                }
                errors.append(error_dict)

routes = pd.DataFrame(routes)
routes.to_csv("trasy_PR_kancelare_2025_11_10-14.csv", sep=';', na_rep='', index= False)

if errors:
    with open (f'trasy_PR_kancelare_2025_11_10-14.json', "w", encoding="UTF-8") as output_file:
        json.dump(errors, output_file, ensure_ascii=False, indent=4)
    errors = pd.DataFrame(errors)
    errors.to_csv(f'trasy_PR_kancelare_2025_11_10-14_errors.csv', sep=';', na_rep='', index= False)

0, Letňany, Brumlovka, 2025-11-10, 06:30
1, Letňany, Karlín, 2025-11-10, 06:30
2, Letňany, Smíchov, 2025-11-10, 06:30
3, Chodov, Brumlovka, 2025-11-10, 06:30
4, Chodov, Karlín, 2025-11-10, 06:30
5, Chodov, Smíchov, 2025-11-10, 06:30
6, Parkoviště P + R - Černý Most I, Brumlovka, 2025-11-10, 06:30
7, Parkoviště P + R - Černý Most I, Karlín, 2025-11-10, 06:30
8, Parkoviště P + R - Černý Most I, Smíchov, 2025-11-10, 06:30
9, Nádraží Braník, Brumlovka, 2025-11-10, 06:30
10, Nádraží Braník, Karlín, 2025-11-10, 06:30
11, Nádraží Braník, Smíchov, 2025-11-10, 06:30
12, Zličín I, Brumlovka, 2025-11-10, 06:30
13, Zličín I, Karlín, 2025-11-10, 06:30
14, Zličín I, Smíchov, 2025-11-10, 06:30
0, Letňany, Brumlovka, 2025-11-10, 07:30
1, Letňany, Karlín, 2025-11-10, 07:30
2, Letňany, Smíchov, 2025-11-10, 07:30
3, Chodov, Brumlovka, 2025-11-10, 07:30
4, Chodov, Karlín, 2025-11-10, 07:30
5, Chodov, Smíchov, 2025-11-10, 07:30
6, Parkoviště P + R - Černý Most I, Brumlovka, 2025-11-10, 07:30
7, Parkoviště 

# Obce - kanceláře

In [ ]:
obce = pd.read_csv("DATA/Obce_kancelare_oprava.csv")

# time = Times[0]
routes = []
rail_routes = []

errors = []

limit_time_Prague = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)
limit_time_not_Prague = 20*60 # 20 min, omezení mimo Prahu

for date in Dates:
    for time in Times[:3]:
        for _, row in obce.iterrows():
            # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
            from_lat, from_lon, to_lat, to_lon, spicka = direction_decide(time, row["souradnice_x"], row["souradnice_y"], row["X_KANCELAR"], row["Y_KANCELAR"])

            try:    
                payload = {
                    "query": query,
                    "variables": {
                        "from_lat": float(from_lat),
                        "from_lon": float(from_lon),
                        "to_lat": float(to_lat),
                        "to_lon": float(to_lon),
                        "Date": date,
                        "Time": time,
                        "window": 3600,
                        "NumInt": 5
                        },
                    "operationName": "Parking"
                    }
                        
                # volání API
                response = requests.post(url, headers=headers, json=payload)
                data = response.json()

                # ------------------------TRANSFORMACE-------------------------
                print(f'{row["OBEC_NAZEV"]} - {row["KANCELAR"]}')

                # pomocné proměnné 

                duration_fast = duration(data)
                rail_route_fast = None 
                leg_num_low = leg_num(data)

                fast_route = {}
                leg_low_route = {}
                rail_route = {}

                for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
                    all_coords = build_coordination_string(iteration)
                    geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

                    itinerary = build_itinerary(iteration)

                    transport_lines, transport_vehicles = build_transport_list(iteration)
                    # transports - konkrétní linky
                    # transport_vehicles - typy dopravních prostředků
                    transfers = transfer_num(iteration)
                            
                    # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
                    time_walk_start = start_walk_time(iteration)
                    time_walk_end = end_walk_time(iteration)

                    # ------------ FAST ------------------------------
                    # rozhodovací bloky - rozdělení na nejrychlejší a s nejméně přestupy
                    if int(iteration['duration']) < duration_fast or iteration_index == 0:
                        duration_fast = int(iteration['duration'])
                        # fast_transports = transport_lines
                        fast_route = build_route_dict('nejrychlejsi', row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, municipalities=True)
                    
                    # -------------- LOW LEG NUMBER --------------------
                    if len(iteration['legs']) < leg_num_low and transfers > 0:
                        # počet kroků, kde je více než 2 vynechá iterace, kde je jen chůze

                        # protože rozlišuji chůzi v Praze a v obci, je třeba nejdřív rozdělit směr
                        if spicka == "ranni": # trasa obce -> kancelare
                            # takže time_walk_start - má limit_time_not_Prague
                            if time_walk_start <= limit_time_not_Prague and time_walk_end <= limit_time_Prague: 
                                # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                                leg_num_low = len(iteration['legs'])
                                leg_low_route = build_route_dict("nejmene prestupu", row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, municipalities=True)
                        else:
                            # tady je to obrácene, tedy time_walk_start má limit_time_Prague
                            if time_walk_start <= limit_time_Prague and time_walk_end <= limit_time_not_Prague: 
                                # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                                leg_num_low = len(iteration['legs'])
                                leg_low_route = build_route_dict("nejmene prestupu", row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs,municipalities=True)
                        
                    # ------------- RAIL ------------------------------------
                    if "RAIL" in transport_vehicles:
                        if rail_route_fast == None or iteration['duration'] < rail_route_fast:
                            rail_route = build_route_dict("vlak", row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs,municipalities=True)
                            rail_route_fast = iteration['duration']

                if leg_low_route:
                    if fast_route["POCET_PRESTUPU"] == leg_low_route["POCET_PRESTUPU"]: 
                        routes.append(fast_route)
                    else:
                        routes.append(fast_route)
                        routes.append(leg_low_route)
                else:
                        routes.append(fast_route)
                if rail_route:
                    rail_routes.append(rail_route)

            except Exception:
                error_dict = {
                    "OBEC_NAZEV": row["OBEC_NAZEV"],
                    "OKRES_KOD": row["OKRES_KOD"],
                    "KANCELAR": row["KANCELAR"],
                    "DATUM": date,
                    "CAS": time,
                    "from_lat": from_lat,
                    "from_lon":from_lon,
                    "to_lat": to_lat,
                    "to_lon": to_lon,
                    "data": data
                }
                errors.append(error_dict)

routes = pd.DataFrame(routes)

adress = f"trasy_obce_{date}"
routes.to_csv(f"{adress}.csv", sep=';', na_rep='', index= False)

rail_routes = pd.DataFrame(rail_routes)
rail_routes.to_csv(f"{adress}_rail.csv", sep=';', na_rep='', index= False)

if errors:
    with open (f'{adress}_errors.json', "w", encoding="UTF-8") as output_file:
        json.dump(errors, output_file, ensure_ascii=False, indent=4)
    errors = pd.DataFrame(errors)
    errors.to_csv(f'{adress}_errors.csv', sep=';', na_rep='', index= False)

Černošice - Brumlovka
Černošice - Karlín
Černošice - Smíchov
Škvorec - Brumlovka
Škvorec - Karlín
Škvorec - Smíchov
Bakov nad Jizerou - Brumlovka
Bakov nad Jizerou - Karlín
Bakov nad Jizerou - Smíchov
Benešov - Brumlovka
Benešov - Karlín
Benešov - Smíchov
Benátky nad Jizerou - Brumlovka
Benátky nad Jizerou - Karlín
Benátky nad Jizerou - Smíchov
Beroun - Brumlovka
Beroun - Karlín
Beroun - Smíchov
Brandýs nad Labem-Stará Boleslav - Brumlovka
Brandýs nad Labem-Stará Boleslav - Karlín
Brandýs nad Labem-Stará Boleslav - Smíchov
Buštěhrad - Brumlovka
Buštěhrad - Karlín
Buštěhrad - Smíchov
Bystřice - Brumlovka
Bystřice - Karlín
Bystřice - Smíchov
Bělá pod Bezdězem - Brumlovka
Bělá pod Bezdězem - Karlín
Bělá pod Bezdězem - Smíchov
Březnice - Brumlovka
Březnice - Karlín
Březnice - Smíchov
Dobrovice - Brumlovka
Dobrovice - Karlín
Dobrovice - Smíchov
Dobřichovice - Brumlovka
Dobřichovice - Karlín
Dobřichovice - Smíchov
Dobříš - Brumlovka
Dobříš - Karlín
Dobříš - Smíchov
Dolní Bousov - Brumlovka
D

# Praha - kanceláře

In [ ]:
# GraphQL query - tady je navíc ještě GPS lokace nástupních zastávek. Když by to ale bylo i ve společném query, nebude patrně žádný problém.
query = '''query Parking (
        $from_lat: Float!,              # definování proměnných, které se vkládají ve "variables"
        $from_lon: Float!, 
        $to_lat: Float!, 
        $to_lon: Float!,
        $Date: String!,
        $Time: String!,
        $window: Long!,
        $NumInt: Int!
        ) 
    {
    plan(
        from: {lat: $from_lat, lon: $from_lon}
        to: {lat: $to_lat, lon: $to_lon}
        date: $Date
        time: $Time                    
        searchWindow: $window           # v jakém časovém okně od zadaného času, bude hledat trasy (v s)
        numItineraries: $NumInt         # počet iterací
        transportModes: [               # definice dopravních prostředků
            {mode: WALK},              
            {mode: BUS},               
            {mode: TRAM}, 
            {mode: RAIL}, 
            {mode: SUBWAY}, 
            {mode: TROLLEYBUS}, 
            {mode: FERRY}
        ]
        ) {
            itineraries {
            start                       # datetime, timezone - Prague
            end                         # datetime, timezone - Prague 
            duration                    # in seconds
            legs {
                mode
                route {
                shortName
                }
                distance
                duration
                from {
                name
                lat
                lon
                }
                to {
                name
                lat
                lon
                }
                legGeometry { 
                length
                points
                }
            }
            }
        }
        }
'''
row_number_1 = 0 # limity pro slicing tabulky
row_number_2 = 10000

Praha_Kancelare = pd.read_csv("DATA/API_zdroje/Praha-kancelare.csv").iloc[row_number_1:row_number_2]


limit_time = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)
limit_time_walk = 20*60 # V Praze jsou nemovitosti i v docházkové vzdálenosti a někteří lidé si pojedou ty 2 zastávky, ale někteří radši půjdou pěšky. Ale pochybuji, že déle jak 20 min

date = Dates[2]
time = "7:00"
print(date)


#for date in Dates[0]:
routes = []  
errors = []
for _, row in Praha_Kancelare.iterrows():

    # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
    from_lat, from_lon, to_lat, to_lon, spicka = direction_decide(time, row["data_gpsCoord_lat"], row["data_gpsCoord_lon"], row["KANCELAR_X"], row["KANCELAR_Y"])
    
    try:
        payload = {
            "query": query,
            "variables": {
                "from_lat": float(from_lat),
                "from_lon": float(from_lon),
                "to_lat": float(to_lat),
                "to_lon": float(to_lon),
                "Date": date,
                "Time": time,
                "window": 1800,
                "NumInt": 5
                },
            "operationName": "Parking"
            }
            

            # volání API
        response = requests.post(url, headers=headers, json=payload, timeout=30)
        data = response.json()

        print(f'{_}, {row["id"]},{row["data_address"]} - {row['KANCELAR']} ')

        # ------------------------TRANSFORMACE-------------------------

        # pomocné proměnné
                    
        duration_fast = duration(data)
        leg_num_low = leg_num(data)
        duration_walk = float('inf') # nastaveno na nekonečno

        fast_route = {}
        leg_low_route = {}
        walk_route = {}

        for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
            all_coords = build_coordination_string(iteration)
            geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

            itinerary = build_itinerary(iteration)

            transport_lines, transport_vehicles = build_transport_list(iteration)
            # transports - konkrétní linky
            # transport_vehicles - typy dopravních prostředků
            transfers = transfer_num(iteration)
                        
            # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
            time_walk_start = start_walk_time(iteration)
            time_walk_end = end_walk_time(iteration)

            # ------------ FAST ----------
            # rozhodovací bloky - rozdělení na nejrychlejší a s nejméně přestupy
            if int(iteration['duration']) < duration_fast or iteration_index == 0:
                duration_fast = int(iteration['duration'])
                fast_route = build_route_dict('nejrychlejsi', row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, Prague=True)
                    
            # -------------- LOW LEG NUMBER --------------------
            # tady je to jedno, ale v případě obcí je třeba ošetřit cestu na vlak (ta může být delší a pak se to dá oseknout)
            if len(iteration['legs']) < leg_num_low and transfers > 0:
                # počet kroků, kde je více než 2 vynechá iterace, kde je jen chůze
                    
                if time_walk_start <= limit_time and time_walk_end <= limit_time: 
                    # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                    leg_num_low = len(iteration['legs'])
                    leg_low_route = build_route_dict ("nejmene prestupu", row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs,Prague=True)
                
            # walk
            if len(iteration['legs']) == 1 and iteration["duration"] <= limit_time_walk:
                if int(iteration['duration']) < duration_walk:
                    duration_walk = int(iteration['duration'])
                    walk_route = build_route_dict('chuze', row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, Prague=True)

        if leg_low_route:
            if fast_route["POCET_PRESTUPU"] == leg_low_route["POCET_PRESTUPU"]: 
                routes.append(fast_route)
            else:
                routes.append(fast_route)
                routes.append(leg_low_route)
        else:
            routes.append(fast_route)

        if walk_route:
            if fast_route["POCET_PRESTUPU"] != walk_route["POCET_PRESTUPU"]: 
                routes.append(walk_route)

                
    except Exception:
        error_dict = {
            "id": row["id"],
            "data_address": row["data_address"],
            "KANCELAR": row["KANCELAR"],
            "DATUM": date,
            "CAS": time,
            "from_lat": from_lat,
            "from_lon":from_lon,
            "to_lat": to_lat,
            "to_lon": to_lon,
            "data": data
        }
        errors.append(error_dict)

routes = pd.DataFrame(routes)
address = f"DATA_HD/trasy_PRAHA_{date}_{row_number_1}-{row_number_2}"
routes.to_csv(f'{address}.csv', sep=';', na_rep='', index= False)

if errors:
    with open (f'{address}_errors.json', "w", encoding="UTF-8") as output_file:
        json.dump(errors, output_file, ensure_ascii=False, indent=4)
    errors = pd.DataFrame(errors)
    errors.to_csv(f'{address}_errors.csv', sep=';', na_rep='', index= False)

numbers = [date, row_number_1, row_number_2]
address_2 = f"DATA_HD/trasy_PRAHA_kontrola"
with open(f'{address_2}.txt', "a", encoding="UTF-8") as output_file_2:
    print(numbers, file=output_file_2)

2025-11-05
50000, 668e6035cf63967a8ac29918,Nad lesním divadlem, Praha 4 - Braník - Smíchov 
50001, 6807525ed6e2288a7192b0cd,Bínova, Praha 8 - Střížkov - Brumlovka 
50002, 6807525ed6e2288a7192b0cd,Bínova, Praha 8 - Střížkov - Karlín 
50003, 6807525ed6e2288a7192b0cd,Bínova, Praha 8 - Střížkov - Smíchov 
50004, 67b9e87f92ccc45937d098fb,Anny Drabíkové, Praha - Háje - Brumlovka 
50005, 67b9e87f92ccc45937d098fb,Anny Drabíkové, Praha - Háje - Karlín 
50006, 67b9e87f92ccc45937d098fb,Anny Drabíkové, Praha - Háje - Smíchov 
50007, 6863c446511ec6e4d7c94cd2,Rohanské nábřeží, Praha 8 - Karlín - Brumlovka 
50008, 6863c446511ec6e4d7c94cd2,Rohanské nábřeží, Praha 8 - Karlín - Karlín 
50009, 6863c446511ec6e4d7c94cd2,Rohanské nábřeží, Praha 8 - Karlín - Smíchov 
50010, 65aa40c4c97da6715486f6ad,Na vysoké I, Praha 5 - Brumlovka 
50011, 65aa40c4c97da6715486f6ad,Na vysoké I, Praha 5 - Karlín 
50012, 65aa40c4c97da6715486f6ad,Na vysoké I, Praha 5 - Smíchov 
50013, 6798fea766a63bef62f97351,Šumberova, Praha 6 -

# Nádraží - kanceláře

In [ ]:
nadrazi = pd.read_csv("NADRAZI_KANCELARE.csv")

# date = Dates[4]

Dates = ["2025-11-03", "2025-11-04", "2025-11-05", "2025-11-06", "2025-11-07"]
# Dates = ["2025-11-10", "2025-11-11", "2025-11-12", "2025-11-13", "2025-11-14"]
Times = ["06:30", "07:30", "08:30", "17:00"] 

# time = Times[0]
routes = []
rail_routes = []

errors = []

not_found_rails = []

limit_time_Prague = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)
limit_time_not_Prague = 20*60 # 20 min, omezení mimo Prahu

for date in Dates:
    for time in Times[:3]:
        for _, row in nadrazi.iterrows():
            # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
            from_lat, from_lon, to_lat, to_lon, spicka = direction_decide(time, row["STOP_LAT"], row["STOP_LON"], row["KANCELAR_LAT"], row["KANCELAR_LON"])

            try:    
                payload = {
                    "query": query,
                    "variables": {
                        "from_lat": float(from_lat),
                        "from_lon": float(from_lon),
                        "to_lat": float(to_lat),
                        "to_lon": float(to_lon),
                        "Date": date,
                        "Time": time,
                        "window": 3600,
                        "NumInt": 5
                        },
                    "operationName": "Parking"
                    }
                        
                # volání API
                response = requests.post(url, headers=headers, json=payload)
                data = response.json()

                # ------------------------TRANSFORMACE-------------------------
                print(f'{date}, {time}: {_}, {row["STOP_NAME"]} - {row["JMENO"]}')

                # pomocné proměnné 

                duration_fast = float('inf')
                duration_fast_walk = float('inf')

                fast_route = {}
                fast_route_walk = {}

                for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
                    all_coords = build_coordination_string(iteration)
                    geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

                    itinerary = build_itinerary(iteration)

                    transport_lines, transport_vehicles = build_transport_list(iteration)
                    # transports - konkrétní linky
                    # transport_vehicles - typy dopravních prostředků
                    transfers = transfer_num(iteration)
                            
                    # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
                    
                    time_walk_end = end_walk_time(iteration)

                    # ------------ RAIL v prvním kroku ---------------
                    if iteration['legs'][0]['mode'] == "RAIL" or iteration['legs'][1]['mode'] == "RAIL":
                        if int(iteration['duration']) < duration_fast:
                            duration_fast = int(iteration['duration'])
                            fast_route = build_route_dict('nejrychlejsi', row, "STOP_ID", "JMENO",  date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs)
                        if int(iteration['duration']) < duration_fast_walk and time_walk_end < limit_time_Prague:
                                duration_fast_walk = int(iteration['duration'])
                                fast_route_walk = build_route_dict('nejrychlejsi', row, "STOP_ID", "JMENO",  date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs)
                        
                
                if fast_route_walk:
                    routes.append(fast_route_walk)
                elif fast_route:       
                    routes.append(fast_route)
                else:
                    not_found_rail = {
                        "STOP_ID": row["STOP_ID"],
                        "STOP_NAME": row["STOP_NAME"],
                        "KANCELAR": row["JMENO"],
                        "DATUM": date,
                        "CAS": time,
                    }
                    not_found_rails.append(not_found_rail)

            except Exception:
                error_dict = {
                    "STOP_ID": row["STOP_ID"],
                    "STOP_NAME": row["STOP_NAME"],
                    "KANCELAR": row["JMENO"],
                    "DATUM": date,
                    "CAS": time,
                    "from_lat": from_lat,
                    "from_lon":from_lon,
                    "to_lat": to_lat,
                    "to_lon": to_lon,
                    "data": data
                }
                errors.append(error_dict)

routes = pd.DataFrame(routes)
# adress = f"DATA_HD/trasy_obce_{date}T{time[:2]}"
adress = f"trasy_nadrazi_kancelare_03-07"
routes.to_csv(f"{adress}.csv", sep=';', na_rep='', index= False)

if errors:
    errors = pd.DataFrame(errors)
    errors.to_csv(f'{adress}_errors.csv', sep=';', na_rep='', index= False)

if not_found_rails:
    not_found_rails = pd.DataFrame(not_found_rails)
    not_found_rails.to_csv(f'{adress}_no_rails.csv', sep=';', na_rep='', index= False)


2025-11-03, 06:30: 0, Bakov nad Jizerou město - Brumlovka
2025-11-03, 06:30: 1, Bakov nad Jizerou město - Karlín
2025-11-03, 06:30: 2, Bakov nad Jizerou město - Smíchov
2025-11-03, 06:30: 3, Praha-Sedlec - Brumlovka
2025-11-03, 06:30: 4, Praha-Sedlec - Karlín
2025-11-03, 06:30: 5, Praha-Sedlec - Smíchov
2025-11-03, 06:30: 6, Třebestovice - Brumlovka
2025-11-03, 06:30: 7, Třebestovice - Karlín
2025-11-03, 06:30: 8, Třebestovice - Smíchov
2025-11-03, 06:30: 9, Lašovice - Brumlovka
2025-11-03, 06:30: 10, Lašovice - Karlín
2025-11-03, 06:30: 11, Lašovice - Smíchov
2025-11-03, 06:30: 12, Kosobody - Brumlovka
2025-11-03, 06:30: 13, Kosobody - Karlín
2025-11-03, 06:30: 14, Kosobody - Smíchov
2025-11-03, 06:30: 15, Kojovice - Brumlovka
2025-11-03, 06:30: 16, Kojovice - Karlín
2025-11-03, 06:30: 17, Kojovice - Smíchov
2025-11-03, 06:30: 18, Rudná zast. - Brumlovka
2025-11-03, 06:30: 19, Rudná zast. - Karlín
2025-11-03, 06:30: 20, Rudná zast. - Smíchov
2025-11-03, 06:30: 21, Domoušice - Brumlovk